In [ ]:
import pandas as pd
import numpy as np
import ast

# Embeddings

In [ ]:
movies_metadata = pd.read_csv('/content/drive/MyDrive/Datasets/mrs/movies_metadata.csv', low_memory=False)
movies_keywords = pd.read_csv('/content/drive/MyDrive/Datasets/mrs/keywords.csv', low_memory=False)
movies_credits = pd.read_csv('/content/drive/MyDrive/Datasets/mrs/credits.csv', low_memory=False)

In [ ]:
movies_metadata.shape, movies_keywords.shape, movies_credits.shape

((45466, 24), (46419, 2), (45476, 3))

In [ ]:
movies_metadata = movies_metadata.drop_duplicates(subset='id')
movies_keywords = movies_keywords.drop_duplicates(subset='id')
movies_credits = movies_credits.drop_duplicates(subset='id')

In [ ]:
movies_metadata.shape, movies_keywords.shape, movies_credits.shape

((45436, 24), (45432, 2), (45432, 3))

##Metadata

In [ ]:
def getGenres(obj):
  a = []
  for o in ast.literal_eval(obj):
    try:
      a.append(f"\"{o['name']}\"");
    except:
      pass
  return ', '.join(a)

def getProductionCompanies(obj):
  try:
    a = []
    for o in ast.literal_eval(obj):
      try:
        a.append(f"\"{o['name']}\"");
      except:
        pass
    return ', '.join(a)
  except:
    return 'No production companies'

def getID(obj):
  try:
    return int(obj)
  except:
    return 0

In [ ]:
movies_metadata.loc[:, 'overview'] = movies_metadata['overview'].fillna('No overview')
movies_metadata.loc[:, 'tagline'] = movies_metadata['tagline'].fillna('No tagline')
movies_metadata.loc[:, 'title'] = movies_metadata['title'].fillna('No title')
movies_metadata.loc[:, 'release_date'] = movies_metadata['release_date'].fillna('No date provided')
movies_metadata.loc[:, 'popularity'] = movies_metadata['popularity'].fillna(0)
movies_metadata.loc[:, 'vote_count'] = movies_metadata['vote_count'].fillna(0)
movies_metadata.loc[:, 'vote_average'] = movies_metadata['vote_average'].fillna(0)

movies_metadata.loc[:, 'id'] = movies_metadata['id'].apply(getID)
movies_metadata.id = movies_metadata.id.astype(int)
movies_metadata.loc[:, 'genres'] = movies_metadata['genres'].apply(getGenres)
movies_metadata.loc[:, 'production_companies'] = movies_metadata['production_companies'].apply(getProductionCompanies)


## Keywords

In [ ]:
def getKeywords(obj):
  a = []
  for o in ast.literal_eval(obj):
    try:
      a.append(f"\"{o['name']}\"");
    except:
      pass
  return ', '.join(a)

In [ ]:
movies_keywords.loc[:, 'keywords'] = movies_keywords['keywords'].apply(getKeywords)

## Cast

In [ ]:
def getCast(obj):
  a = []
  for o in ast.literal_eval(obj):
    try:
      a.append(f"\"{o['name']}\"");
    except:
      pass
  return ', '.join(a[:10])

def getCrew(obj):
  a = []
  cnt = 10;
  for o in ast.literal_eval(obj):
    try:
      a.append(f"\"{o['name']}\"");
      cnt = cnt - 1
      if cnt == 0:
        break
    except:
      pass
  return ', '.join(a[:10])

In [ ]:
movies_credits.loc[:, 'cast'] = movies_credits['cast'].apply(getCast)
movies_credits.loc[:, 'crew'] = movies_credits['crew'].apply(getCrew)

## Final

In [ ]:
movies = pd.merge(movies_keywords, movies_credits, on='id', how='inner')

In [ ]:
movies.shape

(45432, 4)

In [ ]:
movies = pd.merge(movies_metadata, movies, on='id', how='inner')

In [ ]:
movies_content = [f'''overview: {movies.overview[i]}
original title: {movies.original_title[i]}
title: {movies.title[i]}
tagline: {movies.tagline[i]}
release_date: {movies.release_date[i]}
popularity: {movies.popularity[i]}
adult: {movies.adult[i]}
vote count: {movies.vote_count[i]}
vote average: {movies.vote_average[i]}
genres: {movies.genres[i]}
production companies: {movies.production_companies[i]}
cast: {movies.cast[i]}
crew: {movies.crew[i]}
keywords: {movies.keywords[i]}'''
for i in range(len(movies))
]

In [ ]:
movies['content'] = movies_content

In [ ]:
movies = movies[['id', 'content']];

## creating embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

/bin/bash: line 1: import: command not found


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = model.encode(movies['content'], show_progress_bar=True)

Batches:   0%|          | 0/1420 [00:00<?, ?it/s]

In [ ]:
embeddings.shape

(45432, 384)

In [ ]:
np.save("embeddings.npy", embeddings)

In [ ]:
movies['id'].values

array([   862,   8844,  15602, ...,  67758, 227506, 461257])

In [ ]:
np.save("semanticIds.npy", movies['id'].values)

# User Rating Based Embeddings

In [ ]:
user_ratings = pd.read_csv('/content/drive/MyDrive/Datasets/mrs/ratings.csv', low_memory=False)
mapping_df = pd.read_csv('/content/drive/MyDrive/Datasets/mrs/links.csv', low_memory=False)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
user_ratings = user_ratings.merge(mapping_df[['movieId', 'tmdbId']], on='movieId')

In [ ]:
user_ratings['tmdbId'] = pd.to_numeric(user_ratings['tmdbId'], errors='coerce').astype('Int64')

In [ ]:
user_ratings.shape

In [ ]:
user_ratings.dropna(inplace=True)

In [ ]:
user_ratings = user_ratings.sort_values(by='timestamp')
user_ratings = user_ratings.drop_duplicates(subset=['userId', 'tmdbId'], keep='last')

In [ ]:
user_ratings.shape

In [ ]:
min_user_ratings = 10
min_movie_ratings = 20
movie_counts = user_ratings['tmdbId'].value_counts()
user_counts = user_ratings['userId'].value_counts()

In [ ]:
user_ratings = user_ratings[
    user_ratings['tmdbId'].isin(movie_counts[movie_counts >= min_movie_ratings].index) &
    user_ratings['userId'].isin(user_counts[user_counts >= min_user_ratings].index)
]

In [ ]:
user_ratings = user_ratings.pivot(index='tmdbId', columns='userId', values='rating').fillna(0)

/tmp/ipython-input-46-3516587451.py:1: PerformanceWarning: The following operation may generate 3919124880 cells in the resulting pandas object.
  user_ratings = user_ratings.pivot(index='tmdbId', columns='userId', values='rating').fillna(0)


In [ ]:
user_ratings = user_ratings[user_ratings.index.isin(movies['id'])]

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

# Convert to sparse matrix for efficiency
sparse_matrix = csr_matrix(user_ratings.values)

In [ ]:
# Apply SVD to reduce dimensions
svd = TruncatedSVD(n_components=100)  # You can tune 50, 100, 200, etc.
user_embeddings = svd.fit_transform(sparse_matrix)

In [ ]:
user_embeddings.shape

(16694, 100)

In [ ]:
np.save("user_embeddings.npy", user_embeddings)

In [ ]:
hybridIds = user_ratings.index.tolist()

In [ ]:
np.save("hybridIds.npy", hybridIds)

#Combining Embeddings

In [ ]:
movie_embeddings = np.load('embeddings.npy')
user_embeddings = np.load('user_embeddings.npy')
semantic_ids = np.load('semanticIds.npy')
hybrid_ids = np.load('hybridIds.npy')

In [ ]:
from sklearn.preprocessing import normalize
movie_embeddings = normalize(movie_embeddings)
user_embeddings = normalize(user_embeddings)

In [ ]:
user_embeddings.shape, movie_embeddings.shape

((16694, 100), (45432, 384))

In [ ]:
hybrid_ids.shape, semantic_ids.shape

((16694,), (45432,))

In [ ]:
movie_df = pd.DataFrame()
movie_df['id'] = semantic_ids
movie_df['movie'] = list(movie_embeddings)

In [ ]:
hybrid_df = pd.DataFrame()
hybrid_df['id'] = hybrid_ids
hybrid_df['user'] = list(user_embeddings)

In [ ]:
hybrid_df = hybrid_df.merge(movie_df, on='id', how='inner')

In [ ]:
hybrid_embeddings = np.concatenate([0.6 * np.stack(hybrid_df['movie'].values), 0.4 * np.stack(hybrid_df['user'].values)], axis = 1)

In [ ]:
hybrid_embeddings.shape

(16694, 484)

In [ ]:
np.save("hybrid_embeddings.npy", hybrid_embeddings)